<a href="https://colab.research.google.com/github/MpRonald/Machine-Learning/blob/main/Parameter_Tunning_Grid_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parameter_Tunning_Grid_Search

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score, KFold

In [3]:
import pickle as pkl
with open('/content/drive/MyDrive/Datasets/credit.pkl', 'rb') as f:
  X_train, y_train, X_test, y_test = pkl.load(f)

In [4]:
X_train.shape, y_train.shape

((1500, 3), (1500,))

In [5]:
X_test.shape, y_test.shape

((500, 3), (500,))

In [6]:
# concat
X_credit = np.concatenate((X_train, X_test), axis=0) # axis=0 concatenate rows
y_credit = np.concatenate((y_train, y_test), axis=0)
X_credit.shape, y_credit.shape

((2000, 3), (2000,))

In [90]:
# function to run Grid Search
def run_grid_search(estimators, param):
  grid = GridSearchCV(estimator=estimators, param_grid=param).fit(X_credit, y_credit)
  print(f"Best Parameter: {grid.best_params_}")
  print(f"Best Index: {grid.best_index_}")
  best_score = round(grid.best_score_,4)
  print(f"Best Score: {best_score}")

# Decision Tree Classifier

In [91]:
# parameters to classifier
parameters_D = {
      'criterion':['gini', 'entropy'],
      'splitter':['best', 'random'],
      'min_samples_split':[2,5,10],
      'min_samples_leaf':[2,6,10]}

In [92]:
run_grid_search(DecisionTreeClassifier(), parameters_D)

Best Parameter: {'criterion': 'entropy', 'min_samples_leaf': 2, 'min_samples_split': 2, 'splitter': 'best'}
Best Index: 18
Best Score: 0.9835


# Random Forest

In [14]:
# parameters to classifier
parameters_R = {
      'criterion':['gini', 'entropy'],
      'n_estimators':[10, 50, 100, 200],
      'min_samples_split':[2,5,10],
      'min_samples_leaf':[2,6,10]}

In [17]:
result_RF = GridSearchCV(RandomForestClassifier(), param_grid = parameters_R).fit(X_credit, y_credit)
best_RF = result_RF.best_score_
best_RF

0.986

# KNN

In [18]:
parameters_K = {
    'n_neighbors':[3, 5, 10, 20, 35],
    'p':[1,2]}

In [19]:
run_grid_search(KNeighborsClassifier(), parameters_K)

Best Parameter: {'n_neighbors': 35, 'p': 2}
Best Index: 9
Best Score: 0.9815


In [20]:
result_knn = GridSearchCV(KNeighborsClassifier(), param_grid = parameters_K).fit(X_credit, y_credit)
best_knn = result_knn.best_score_
best_knn

0.9814999999999999

# Logistic Regression

In [21]:
parameters_L ={
    'tol': [0.1, 0.01, 0.001, 0.0001, 0.00001],
    'C': [1.0, 1.5, 2.0, 2.5, 3.0],
    'solver': ['lbfgs', 'sag', 'saga']
}

In [22]:
run_grid_search(LogisticRegression(), parameters_L)

Best Parameter: {'C': 1.0, 'solver': 'lbfgs', 'tol': 0.1}
Best Index: 0
Best Score: 0.9485


In [29]:
result_lr = GridSearchCV(LogisticRegression(), param_grid = parameters_L).fit(X_credit, y_credit)
best_lr = result_lr.best_score_
best_lr

0.9490000000000001

# SVM

In [24]:
parameters_S = {
    'tol': [0.1, 0.01, 0.001, 0.0001, 0.00001],
    'C': [1.0, 1.5, 2.0, 2.5, 3.0],
    'kernel':['rbf', 'linear', 'poly', 'sigmoid']}

In [25]:
run_grid_search(SVC(), parameters_S)

Best Parameter: {'C': 3.0, 'kernel': 'rbf', 'tol': 0.1}
Best Index: 80
Best Score: 0.984


In [27]:
result_svm = GridSearchCV(SVC(), param_grid = parameters_S).fit(X_credit, y_credit)
best_svm = result_svm.best_score_
best_svm

0.984

# Neural Netwoks

In [30]:
parameters_N = {
    'activation':['relu', 'logistic', 'tahn'],
    'solver':['adam', 'sgd'],
    'batch_size':[10, 42]}

In [ ]:
run_grid_search(MLPClassifier(), parameters_N)

In [ ]:
result_nn = GridSearchCV(MLPClassifier(), param_grid = parameters_N).fit(X_credit, y_credit)
best_nn = result_nn.best_score_
best_nn

# Cross Validation

In [33]:
def cross_validation(algorithm, X, y):
  list_result = []
  for i in range(30):
    kfold = KFold(n_splits=10, shuffle=True, random_state=i)
    
    clf = algorithm
    scores = cross_val_score(clf, X, y, cv = kfold)
    list_result.append(scores.mean())
  print(f'Mean of : {list_result}')

In [34]:
list_algorithm = [DecisionTreeClassifier(), RandomForestClassifier(),
                  LogisticRegression(), SVC(), MLPClassifier()]

In [ ]:
for i in list_algorithm:
  print(i)
  cross_validation(i, X_credit, y_credit)

# Loading a trained classifier

In [94]:
neural = MLPClassifier(activation='relu', batch_size=56, solver='adam').fit(X_credit, y_credit)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


In [96]:
decision_tree = DecisionTreeClassifier(criterion='entropy', min_samples_leaf=1,
                                       min_samples_split=5, splitter='best').fit(X_credit, y_credit)

In [98]:
svm = SVC(C=2.0, kernel='rbf').fit(X_credit, y_credit)

In [101]:
# creating files
import pickle as pkl
pkl.dump(neural, open('neural_final.sav', 'wb'))
pkl.dump(decision_tree, open('tree_final.sav', 'wb'))
pkl.dump(svm, open('svm_final.sav', 'wb'))

In [103]:
# loading
neural = pkl.load(open('neural_final.sav', 'rb'))
tree = pkl.load(open('tree_final.sav', 'rb'))
svm = pkl.load(open('svm_final.sav', 'rb'))

In [117]:
new_register = X_credit[1999]

In [118]:
# prediction
new_register = new_register.reshape(1,-1)

In [120]:
# neural network
output = neural.predict(new_register)
if output == 0:
  print('Crédito Consedido!')
else:
  print('Crédito Negado!')

Crédito Negado!


In [121]:
# decision tree
output = tree.predict(new_register)
if output == 0:
  print('Crédito Consedido!')
else:
  print('Crédito Negado!')

Crédito Negado!


In [122]:
# svm
output = svm.predict(new_register)
if output == 0:
  print('Crédito Consedido!')
else:
  print('Crédito Negado!')

Crédito Negado!
